# PJ's Variable reduction method (Interior)

`1712.04.Aug.20`

PJ and Dr. Ning wrote a paper on the [Variable Reduction](https://scholarsarchive.byu.edu/facpub/3567/) method that was very good. I'm applying it to BYU's IEA participation, and it comes in two parts:

1) parameterize some turbines along the edge

2) parameterize interior turbines in a specific way

(1) Was completed in the previous two notebooks (numbers 04 & 05). This notebook will code up part (1) in Julia. PJ wrote his original code in Python, so I'll need to translate that. It's not well commented, but we did a Video Call today and he explained each piece. I'll include it here below.

### PJ's Interior Variable Reduction Code
`04.04.Aug.20`

``` python
#fill an arbirary boundary with a baseline grid
def startGrid(nTurbines,boundaryVertices,boundaryNormals,rotate=0.,my=1.):
    xBounds = boundaryVertices[:, 0]
    yBounds = boundaryVertices[:, 1]

    t = 25
    range_x = (max(xBounds)-min(xBounds))
    range_y = (max(yBounds)-min(yBounds))
    d = 1.5*max(np.array([range_x,range_y]))/float(t)
    # rotate = 0.
    nRows = t
    turbs_per_row = np.ones(nRows,dtype=int)*t
    N = np.sum(turbs_per_row)

    # x_start = np.ones(t)*min(xBounds)-range_x*0.5
    x_start = np.zeros(t)
    y0 = min(yBounds)-0.05*range_y

    shear = np.tan(np.deg2rad(20.))*d
    X,Y = grid_param_fortran.makegrid_fortran(N, d, d*my, shear, rotate, turbs_per_row, x_start, y0)

    bounds = arbitraryBoundary(X, Y, boundaryVertices, boundaryNormals)
    num_inside = 0
    for i in range(t**2):
        if min(bounds[i]) >= 0.:
            num_inside += 1

    converged = False
    step = 0.25
    print 'setting up base grid'
    # print nTurbines
    while converged == False:
        while num_inside < nTurbines:
            num_inside,d = startLoop(N,d,my,step,-1.,rotate,turbs_per_row,x_start,y0,boundaryVertices,boundaryNormals,t)
        while num_inside >= nTurbines:
            num_inside,d = startLoop(N,d,my,step,1.,rotate,turbs_per_row,x_start,y0,boundaryVertices,boundaryNormals,t)

        if step < 1.E-8:
            converged = True
        else:
            step = step/2.
        # print step

    print 'final convergence'
    num_inside,d = startLoop(N,d,my,step,-1.,rotate,turbs_per_row,x_start,y0,boundaryVertices,boundaryNormals,t)
    small_step = step/100.
    while num_inside >= nTurbines:
        num_inside,d = startLoop(N,d,my,small_step,1.,rotate,turbs_per_row,x_start,y0,boundaryVertices,boundaryNormals,t)

    while num_inside < nTurbines:
        d = d-d*small_step
        shear = np.tan(np.deg2rad(20.))*d
        X,Y = grid_param_fortran.makegrid_fortran(N, d, d*my, shear, rotate, turbs_per_row, x_start, y0)
        bounds = arbitraryBoundary(X, Y, boundaryVertices, boundaryNormals)
        num_inside = 0
        for i in range(t**2):
            if min(bounds[i]) > 0.:
                num_inside += 1

    x = np.zeros(nTurbines)
    y = np.zeros(nTurbines)
    turb = 0
    index = 0
    while turb < nTurbines:
        if min(bounds[index]) >= 0.:
                x[turb] = X[index]
                y[turb] = Y[index]
                turb += 1
        index += 1

    dx = d
    dy = d*my
    turbineX = np.cos(-np.deg2rad(rotate))*x - np.sin(-np.deg2rad(rotate))*y
    turbineY = np.sin(-np.deg2rad(rotate))*x + np.cos(-np.deg2rad(rotate))*y
    y0 = np.min(turbineY)
    turbs_per_row = np.array([1])
    for i in range(nTurbines-1):
        if turbineY[i+1] < turbineY[i]+1E-4 and turbineY[i+1] > turbineY[i]-1E-4:
            turbs_per_row[-1] += 1
        else:
            turbs_per_row = np.append(turbs_per_row,1)

    shear = dy*np.tan(np.deg2rad(20.))
    nRows = len(turbs_per_row)
    x_start = np.zeros(nRows)
    x_start[0] = 0.0
    index = 1
    # for i in range(nTurbines-1):
    #     if not (turbineY[i+1] < turbineY[i]+1.E-4 and turbineY[i+1] > turbineY[i]-1.E-4):
    #         xs = turbineX[i+1]
    #         x_start[index] = (xs-shear-turbineX[sum(turbs_per_row[0:index-1])])/dx
    #         index += 1
    for i in range(nRows-1):
        x_start[i+1] = x_start[i] + (turbineX[sum(turbs_per_row[0:i+1])] - shear - turbineX[sum(turbs_per_row[0:i])])/dx

    # plt.figure(1)
    # for i in range(nTurbines):
    #     circ = plt.Circle((turbineX[i],turbineY[i]),130./2.,alpha=0.5,color='blue')
    #     plt.gca().add_patch(circ)
    #     circ = plt.Circle((x[i],y[i]),130./2.,alpha=0.5,color='red')
    #     plt.gca().add_patch(circ)
    # bx = boundaryVertices[:,0]
    # by = boundaryVertices[:,1]
    # bx = np.append(bx,bx[0])
    # by = np.append(by,by[0])
    # plt.plot(bx,by,'--k')
    # plt.axis('equal')
    # plt.show()

    return dx,dy,shear,rotate,turbs_per_row,x_start,y0
```